# OANC Commissioner List

Download a list of commissioners from the Office of Advisory Neighborhood Commissions and compare it against the OpenANC list.

In [1]:
import os
os.chdir('../')

In [2]:
import time
import requests
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz

import config
from scripts.common import hash_dataframe, current_timestamp, validate_smd_ids
from scripts.data_transformations import list_commissioners

pd.set_option('display.max_rows', 500)

In [3]:
ancs = pd.read_csv('data/ancs.csv')
ancs = ancs[ancs.redistricting_year == config.current_redistricting_year].copy()

In [4]:
comm_dict = {}

for idx, anc in tqdm(ancs.iterrows(), total=len(ancs)):
    r = requests.get(anc.dc_oanc_link)
    
    table_list = pd.read_html(r.text, converters={'SMD': str})

    if 'SMD' in table_list[0].columns:
        comm_dict[anc.anc_id] = table_list[0].copy()
    elif anc.anc_id == 'anc_5F_2022':
        # First row of this dataframe is the column names and should be fixed then dropped
        comm_dict[anc.anc_id] = table_list[0].copy()
        comm_dict[anc.anc_id].columns = comm_dict[anc.anc_id].iloc[0]
        comm_dict[anc.anc_id].drop(0, inplace=True)
    else:
        print(f'Warning: ANC {anc.anc_id} table not present on OANC site.')
        print(table_list)
    
    time.sleep(2)

100%|███████████████████████████████████████████| 46/46 [01:45<00:00,  2.30s/it]


In [5]:
comm_official = pd.concat(comm_dict).reset_index().rename(columns={'level_0': 'anc_id'}).drop(columns='level_1')

In [6]:
comm_official['smd_id'] = 'smd_2022_' + comm_official['SMD']

# Fix some mangled SMD IDs
comm_official['smd_id'] = (
    comm_official['smd_id']
    .str.replace('smd_2022_3G', 'smd_2022_3/4G')
    .str.replace('smd_2022_7B02l', 'smd_2022_7B02')
    .str.replace('smd_2022_6/8F', 'smd_2022_8F')
)


In [7]:
# Confirm that district list is correct
validate_smd_ids(comm_official)
assert len(comm_official) == 345

In [8]:
comm_official['is_vacant'] = comm_official.Name.str.lower()== 'vacant'
comm_official['is_chairperson'] = comm_official.Name.str.lower().str.contains('chairperson|chairpeson')

In [9]:
strings_to_remove_from_names = [
    ' Chairperson'
    , ' Chairpeson'
    , ' Vice Chair/Treasurer'
    , ' Vice Chair'
    , ' Corresponding Secretary'
    , ' Recording Secretary'
    , ' Secretary'
    , ' /Secretary'
    , ' Seretary'
    , ' Parlimentarian'
    , ' Sargent-at-arms'
    , ' Treasurer'
    , ' Vice-Chairperson'
]

comm_official['oanc_name'] = comm_official['Name']

for s in strings_to_remove_from_names:
    comm_official['oanc_name'] = comm_official['oanc_name'].str.replace(s, '')

comm_official['oanc_name'] = comm_official['oanc_name'].str.strip()

In [10]:
comm_official['oanc_hash_id'] = hash_dataframe(comm_official, ['SMD', 'oanc_name'])

In [11]:
# Compare official to OpenANC
comm_openanc = list_commissioners(status='current')
comm = pd.merge(comm_official, comm_openanc, how='left', on='smd_id')

comm['oanc_name'] = comm['oanc_name'].fillna('Vacant')
comm['openanc_name'] = comm['commissioner_name'].fillna('Vacant')
comm['name_score'] = comm.apply(lambda x: fuzz.ratio(x.oanc_name, x.openanc_name), axis=1)

In [12]:
"""
0 = the district is not vacant according to both sources (good)
1 = the district is vacant according to one source (bad)
2 = the district is vacant according to both sources (good)
"""
comm['num_sources_say_vacant'] = (comm[['oanc_name', 'openanc_name']] == 'Vacant').sum(axis=1)
comm.loc[comm['num_sources_say_vacant'] == 1, 'name_score'] = 0

In [13]:
# Evaluate name mismatches
comm.loc[comm.name_score < 100, ['smd_id', 'oanc_name', 'openanc_name', 'name_score']].sort_values(
    by=['name_score', 'smd_id']
)

,smd_id,oanc_name,openanc_name,name_score
45,smd_2022_2A04,Ed Comer,Vacant,0
54,smd_2022_2B04,Vacant,China Dickerson,0
72,smd_2022_2E07,Daniel Chao,Vacant,0
181,smd_2022_5A09,Shelagh Bocoum,Vacant,0
305,smd_2022_7F08,Shameka Hayes (proxy Danjuna Gaskins),Shameka Hayes,52
82,smd_2022_2G01,"Anthony ""Tony"" Brown",Tony Brown,67
325,smd_2022_8C06,Robbie Woodland,"Robbie ""The Advocate"" Woodland",67
333,smd_2022_8D06,Wendy Hamilton,"Wendy ""Hope Dealer"" Hamilton",67
200,smd_2022_5D05,Salvador Sauceda-Guzman,"Salvador ""The Commissioner"" Sauceda-Guzman",71
341,smd_2022_8E06,Dolores Bryant,"Dolores ""Miracle"" Bryant",74


In [14]:
columns_to_save = [
    'smd_id'
    , 'Name'
    , 'oanc_name'
    , 'is_vacant'
    , 'is_chairperson'
    , 'oanc_hash_id'
]

filename = 'data/oanc/commissioners_{}.csv'.format(current_timestamp().strftime('%Y-%m-%d'))

comm_official[columns_to_save].to_csv(filename, index=False)

In [15]:
# Confirm that there is one chairperson per ANC
comm_official['is_chairperson_int'] = comm_official['is_chairperson'].astype(int)
num_chairs = comm_official.groupby('anc_id').is_chairperson_int.sum()
# num_chairs.sum() == comm_official.anc_id.nunique()

# Total number of chairpeople
# num_chairs.sum()

In [16]:
# Districts with an issue to resolve
comm.loc[comm['num_sources_say_vacant'] == 1, ['smd_id', 'openanc_name']] #.to_clipboard(index=False)

,smd_id,openanc_name
45,smd_2022_2A04,Vacant
54,smd_2022_2B04,China Dickerson
72,smd_2022_2E07,Vacant
181,smd_2022_5A09,Vacant


In [17]:
comm_official

,anc_id,SMD,Name,Address,Phone,Email,smd_id,is_vacant,is_chairperson,oanc_name,oanc_hash_id,is_chairperson_int
0,anc_1A_2022,1A01,Jaspal Bhatia,"Washington, DC 20009",NaN,[email protected],smd_2022_1A01,False,False,Jaspal Bhatia,e102c251d2d896949688627f835714814009458ae6d4a0...,0
1,anc_1A_2022,1A02,Dieter Lehmann Morales Chairperson,"3460 14th Street NW Washington, DC 20010",202-780-7198,[email protected],smd_2022_1A02,False,True,Dieter Lehmann Morales,a8c58f17d6df64440f9a97bf8d89f1455bc71c0692a201...,1
2,anc_1A_2022,1A03,Carlo Perri,"1400 Irving Street NW Washington, DC 20010",NaN,[email protected],smd_2022_1A03,False,False,Carlo Perri,6c8bcac8a8928bb2b2cc83dff1cf4bc5b08faacb9eab05...,0
3,anc_1A_2022,1A04,Jeremy Sherman Vice Chair,"Washington, DC 20010",443-218-2298,[email protected],smd_2022_1A04,False,False,Jeremy Sherman,622d3a5d934b451a9e68d5fc095967b3a0f92306c11f17...,0
4,anc_1A_2022,1A05,Stephen Coleman Kenny,"Washington, DC 20010",202-618-0305,[email protected],smd_2022_1A05,False,False,Stephen Coleman Kenny,ce27ae0bceb0130d130e6a00069265ed5c0326b2f3b858...,0
5,anc_1A_2022,1A06,Anthony Thomas-Davis,"1390 Kenyon Street NW Washington, DC 20010",202-855-8550,[email protected],smd_2022_1A06,False,False,Anthony Thomas-Davis,95ca1a2369cc2c19da0d69f2bb6d4f01a489f44be084db...,0
6,anc_1A_2022,1A07,Mukta Ghorpadey Secretary,"Washington, DC 20010",202-813-0986,[email protected],smd_2022_1A07,False,False,Mukta Ghorpadey,89b5b4fb4376addc23a80ff5a094c825e53c3c04042c21...,0
7,anc_1A_2022,1A08,David Segall,"Washington, DC 20009",NaN,[email protected],smd_2022_1A08,False,False,David Segall,f689a64e52e306a6d2a830b093c64758e734144c7537dc...,0
8,anc_1A_2022,1A09,James A. Turner Treasurer,"Washington, DC 20009",NaN,[email protected],smd_2022_1A09,False,False,James A. Turner,de889d01b78a1db2714076a0386a51c1a731134cd16f6b...,0
9,anc_1A_2022,1A10,Billy Easley,"Washington, DC 20009",202-643-4150,[email protected],smd_2022_1A10,False,False,Billy Easley,1e31fd5eef7242837ed76a44d023856e685fd2e3360094...,0
